# IMPORTS

In [30]:
import pandas as pd
import numpy as np
import os
from decimal import Decimal
import pickle
from pathlib import Path

from sklearn import linear_model
import statsmodels.api as sm

# SETUP

In [31]:
dir_tree_util_path = os.path.join("utils", "dir_tree.py")
exec(open(dir_tree_util_path).read())

# INPUTS

In [32]:
# Paths
path_dados = PROJECT_DIRS["DADOS_DERIVADOS_DIR"]
# path_output = TODO

In [80]:
periodos = [['2010-01','2022-01'], ['2015-09','2022-01']]

In [34]:
path_outputs = PROJECT_DIRS["DADOS_DERIVADOS_DIR"]

# CARREGANDO p/MEMORIA OS DADOS

In [35]:
# Dados de tráfego ANTT
df_veqs_comercial = pd.read_parquet(path_dados / 'df_VEQS_COMERCIAL_mensal_ajustado.parquet')
df_veqs_passeio = pd.read_parquet(path_dados / 'df_VEQS_PASSEIO_mensal_ajustado.parquet')

df_periodos = pd.read_parquet(path_dados / 'df_periodos.parquet')

dict_veqs = {'veqs_comercial':df_veqs_comercial,
           'veqs_passeio':df_veqs_passeio}

In [99]:
# Dados de PIB
df_PIB = pd.read_parquet(path_dados / 'PIB-Bacen_mensal.parquet')

# TRATANDO OS DADOS

In [100]:
df_PIB.index = pd.to_datetime(df_PIB['Data'])
df_PIB = df_PIB.drop(columns=['Data'])

# DISPONIBILIDADE DE DADOS

In [101]:
df_periodos

,data_inicial,data_final
concessionaria,,
HOLDING DO SISTEMA RODOVIARIO RIO - SAO PAULO S.A.,2024-01-01,2024-06-03
VIA BRASIL,2023-02-01,2023-12-01
ECORIOMINAS,2022-10-01,2024-06-02
ECOVIAS DO ARAGUAIA,2022-10-01,2024-06-02
RIOSP,2022-03-01,2023-12-01
VIA COSTEIRA,2022-01-01,2024-04-01
ECOVIAS DO CERRADO,2020-11-14,2024-06-02
VIA SUL,2019-01-01,2024-05-03
MSVIA,2015-09-01,2024-05-03


# FUNCOES

In [104]:
def filter_PIB_periodo(df, periodo: list):
    # Convert the start and end periods to datetime and adjust to the first and last days of the month
    start_date = pd.to_datetime(periodo[0]) + pd.offsets.MonthBegin(0)
    end_date = pd.to_datetime(periodo[1]) + pd.offsets.MonthEnd(0)
    
    # Ensure the DataFrame index is in datetime format
    # df.index = pd.to_datetime(df.index)
    
    # Create a range of dates from start_date to end_date, ensuring end-of-month dates
    datas = pd.date_range(start=start_date, end=end_date, freq='ME')
    
    # Filter the DataFrame using the date range
    df = df[df.index.isin(datas)]
    
    # Drop rows with missing values
    df = df.dropna()
    
    return df

In [49]:
def filter_conc_periodo(df_periodos, periodo:list):
    ano_min = min(periodo)
    ano_max = max(periodo)
    anos = [ano for ano in range(ano_min, ano_max + 1)]
    
    start_date = pd.to_datetime(f'{periodo[0]}').date()
    end_date = pd.to_datetime(f'{periodo[1]}').date()
    
    condition = (df_periodos['data_inicial'] <= start_date) & \
                (df_periodos['data_final'] >= end_date)
    
    df_concs = df_periodos[condition].index
    
    return df_concs

In [53]:
def filter_trafego_periodo(df_trafego, df_periodos, periodo:list):
    df_concs = filter_conc_periodo(df_periodos, periodo)
    df_trafego = df_trafego.loc[df_concs]
    meses_drop = [col for col in df_trafego.columns if (col.dt < periodo[0] - 1) or (col.dt > periodo[1])]
    df_trafego = df_trafego.drop(columns=anos_drop)
    df_trafego = df_trafego.T
    
    # for col in df_trafego:
    #     df_trafego[col] = df_trafego[col].pct_change()*100
        
    # df_trafego = df_trafego.loc[periodo[0]:,:]
            
    return df_trafego

In [54]:
def calc_perc_change(df_trafego, periodo):
    for col in df_trafego:
        df_trafego[col] = df_trafego[col].pct_change()*100
    df_trafego = df_trafego.loc[periodo[0]:,:]
    return df_trafego    

In [55]:
filter_trafego_periodo(df_veqs_comercial, df_periodos, periodos)

TypeError: can only concatenate list (not "int") to list

In [43]:
df_veqs_comercial

MES_ANO_f,2010-01-31,2010-02-28,2010-03-31,2010-04-30,2010-05-31,2010-06-30,2010-07-31,2010-08-31,2010-09-30,2010-10-31,...,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31
concessionaria,,,,,,,,,,,,,,,,,,,,,
AUTOPISTA FERNÃO DIAS,6.730919e+06,6.944982e+06,6.896664e+06,7.143619e+06,7.362983e+06,7.498118e+06,7.448221e+06,7.419812e+06,8.401366e+06,8.626226e+06,...,9.677890e+06,9.517082e+06,9.856563e+06,9.976065e+06,1.005934e+07,NaN,NaN,NaN,NaN,NaN
AUTOPISTA FLUMINENSE,1.827125e+06,1.889539e+06,1.922456e+06,1.898345e+06,2.025843e+06,1.986273e+06,2.026375e+06,2.006717e+06,2.015563e+06,2.032654e+06,...,2.033876e+06,2.050933e+06,2.069400e+06,2.121010e+06,2.101672e+06,2.011871e+06,2.101156e+06,7.826938e+05,NaN,NaN
AUTOPISTA LITORAL SUL,5.587079e+06,5.614516e+06,5.624919e+06,5.691009e+06,5.811210e+06,5.967139e+06,5.864482e+06,5.908488e+06,5.876228e+06,5.953641e+06,...,8.214537e+06,8.193822e+06,8.298718e+06,8.414241e+06,8.163534e+06,3.927876e+06,8.929175e+06,8.991046e+06,NaN,NaN
AUTOPISTA PLANALTO SUL,1.580867e+06,1.588734e+06,1.611791e+06,1.590052e+06,1.663020e+06,1.662241e+06,1.631531e+06,1.651547e+06,1.633915e+06,1.643624e+06,...,1.950814e+06,1.850188e+06,2.320172e+06,2.069420e+06,1.980880e+06,1.892138e+06,2.259548e+06,2.260831e+06,NaN,NaN
AUTOPISTA REGIS BITTENCOURT,8.906674e+06,8.979151e+06,9.389727e+06,9.441370e+06,9.574411e+06,9.677052e+06,9.645862e+06,9.697287e+06,9.771047e+06,9.759458e+06,...,1.100824e+07,1.087130e+07,1.102111e+07,1.118669e+07,1.108748e+07,1.161571e+07,NaN,NaN,NaN,NaN
CONCEBRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.528958e+06,5.537065e+06,5.536905e+06,5.503516e+06,5.453874e+06,5.534546e+06,5.536890e+06,5.474847e+06,5.585717e+06,5.409117e+06
CONCEPA,1.170518e+06,1.186067e+06,1.190471e+06,1.156099e+06,1.161900e+06,1.164430e+06,1.145950e+06,1.158245e+06,1.168258e+06,1.187953e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CONCER,1.005628e+06,1.002992e+06,1.072961e+06,1.064589e+06,1.091544e+06,1.097190e+06,1.098327e+06,1.099072e+06,1.097371e+06,1.110259e+06,...,9.688302e+05,9.271885e+05,9.661489e+05,9.852802e+05,9.709610e+05,7.751409e+05,9.555370e+05,9.469828e+05,9.968583e+05,9.649733e+05
CRO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.703837e+06,8.760626e+06,8.890841e+06,9.509943e+06,9.287125e+06,6.399106e+06,6.738418e+06,7.128465e+06,8.070433e+06,7.826521e+06


In [21]:
filter_trafego_periodo(df_veqs_comercial, df_periodos, [2010,2023])

AttributeError: 'Timestamp' object has no attribute 'dt'

In [262]:
def create_df_PIB_concs(df_trafego, df_PIB, df_periodos, periodo:list):
    
    X_PIB = pd.DataFrame(filter_PIB_periodo(df_PIB, periodo))
    y_veqs = filter_trafego_periodo(df_trafego, df_periodos, periodo)
    
    df_PIB_concs = X_PIB.join(y_veqs)  
        
    return df_PIB_concs

In [295]:
def regressao(df_y, df_periodos, conc, df_X, periodo, fit_intercept=True):
    y_train = filter_trafego_periodo(df_y, df_periodos, periodo)[conc]
    y_train = calc_perc_change(pd.DataFrame(y_train), periodo)
    X_train = filter_PIB_periodo(df_PIB, periodo)
    
    y_train = pd.DataFrame(y_train)
    X_train = pd.DataFrame(X_train)

    if fit_intercept:
        X_train = sm.add_constant(X_train)
    
    model = sm.OLS(y_train, X_train).fit()
    
    return model

In [384]:
def make_models(df_veqs, df_periodos, df_X, periodo, fit_intercept=True):
    concs = filter_trafego_periodo(df_veqs, df_periodos, periodo).columns
    df_stats = pd.DataFrame(index=concs)
    
    for conc in concs:
        model = regressao(df_veqs, df_periodos, conc, df_X, periodo, fit_intercept=fit_intercept)
        param_name = model.params.index
        df_stats.loc[conc, 'R2'] = model.rsquared
        df_stats.loc[conc, f'{param_name[0]}-coef'] = model.params.iloc[0]
        df_stats.loc[conc, f'{model.pvalues.index[0]}-p-valor'] = model.pvalues.iloc[0]

    return df_stats

# REGRESSÃO

## Periodo 2011-2023

In [389]:
model_11_23 = make_models(df_veqs_comercial, df_periodos, df_PIB, [2011, 2023], fit_intercept=False)
model_11_23

,R2,PIB_real-coef,PIB_real-p-valor
concessionaria,,,
TRANSBRASILIANA,0.187843,0.922658,0.121587
RODOVIA DO AÇO,0.416907,1.411764,0.012624
AUTOPISTA FERNÃO DIAS,0.090191,1.233776,0.296831
AUTOPISTA FLUMINENSE,0.324131,1.742415,0.033583
CONCER,0.415267,1.948134,0.012857
AUTOPISTA REGIS BITTENCOURT,0.478093,1.483594,0.006161
AUTOPISTA PLANALTO SUL,0.545963,2.012901,0.002536
AUTOPISTA LITORAL SUL,0.256490,1.781030,0.064601
ECOSUL,0.237700,2.047181,0.076993


In [391]:
model_11_23['R2'].mean()

np.float64(0.3280651550138982)

In [392]:
model_11_23['PIB_real-p-valor'].mean()

np.float64(0.06975263319666872)

In [395]:
model_11_23['PIB_real-coef'].mean()

np.float64(1.6203835338934016)

## Periodo 2017-2023

In [390]:
model_17_23 = make_models(df_veqs_comercial, df_periodos, df_PIB, [2017, 2023], fit_intercept=False)
model_17_23

,R2,PIB_real-coef,PIB_real-p-valor
concessionaria,,,
MSVIA,0.287888,1.696313,0.170373
CRO,0.448201,1.309613,0.069372
VIA 040,0.342956,1.257074,0.127178
ECOPONTE,0.181272,5.140400,0.292926
CONCEBRA,0.047675,0.353209,0.603427
ECO050,0.080677,0.752466,0.495381
ECO101 CONCESSIONARIA DE RODOVIAS S/A,0.336337,1.268124,0.131821
VIA BAHIA,0.099740,0.376899,0.446040
TRANSBRASILIANA,0.000069,-0.011017,0.984413


In [393]:
model_17_23['R2'].mean()

np.float64(0.253334267863935)

In [394]:
model_17_23['PIB_real-p-valor'].mean()

np.float64(0.30519121532145)

In [396]:
model_17_23['PIB_real-coef'].mean()

np.float64(1.2744039613345777)

# Salvando para excel

In [403]:
with pd.ExcelWriter(path_outputs / 'resultado_modelos_PIB.xlsx') as writer:
    # df_resultados = pd.DataFrame(index=df_periodos.index)
    for p in periodos:
        for key, veq in dict_veqs.items():
            df_resultados = make_models(veq, df_periodos, df_PIB, p, fit_intercept=False)
            sheet_name = f'{p[0]}-{p[1]}-{key}'
            df_resultados.to_excel(writer, sheet_name = sheet_name)        